In [4]:
%env KERAS_BACKEND=tensorflow
%matplotliv inline
import numpy as np
import matplotlib.pyplot as plt

env: KERAS_BACKEND=tensorflow


UsageError: Line magic function `%matplotliv` not found.


In [5]:
from keras.datasets import imdb
from keras.preprocessing import sequence

# param : 取最常出現的10000個字
(x_train, y_train),(x_test, y_test) = imdb.load_data(num_words=10000)

print('train data count : ',len(x_train))
print('test data count : ',len(x_test))

# input data 以出現頻率代替文字
print('input data 99: ',x_train[99])
for i in range(0,10):
    # input data has different input
    print(len(x_train[i]),end=', ')

print('\noutput data: ',y_train[:10])


# 雖然RNN可接受不同長度input, 怕太長用看前100字判斷就好,太短則會補0
x_train =sequence.pad_sequences(x_train,maxlen=100)
x_test = sequence.pad_sequences(x_test,maxlen=100)
print('after preprocess dimension: ',x_train.shape)

train data count :  25000
test data count :  25000
input data 99:  [1, 1230, 3765, 566, 97, 189, 102, 86, 7, 32, 4, 973, 16, 55, 355, 18, 14, 20, 4, 64, 542, 173, 16, 4, 893, 2115, 5376, 250, 39, 8013, 4, 1362, 2, 14, 102, 47, 57, 599, 633, 6, 1317, 2, 8, 6, 189, 20, 57, 206, 57, 116, 5, 57, 836, 82, 6, 1317, 2, 3728, 2, 9, 6, 52, 284, 21, 29, 9, 38, 2245, 5, 1044, 11, 14, 15, 45, 619, 50, 71, 6, 171, 531, 15, 71, 424, 8, 30, 163, 6211, 4, 1629, 189, 212, 102, 5, 57, 31, 1498, 11, 4, 311, 13, 197, 15, 14, 20, 16, 1150, 1479, 5, 13, 161, 990, 692, 5, 1706, 12, 69, 77, 1194, 8, 3245, 2001, 553, 67, 14, 20, 48, 25, 423, 13, 131, 124, 51, 25, 122, 236, 1506, 198, 4, 64, 552, 7, 415, 37, 62, 169, 14, 20, 60, 2602, 629, 5, 615, 14, 9, 8, 25, 1230, 3765, 570, 231, 189, 102, 14, 20, 166, 2039, 168, 40, 2450, 5486, 3298]
218, 189, 141, 550, 147, 43, 123, 562, 233, 130, 
output data:  [1 0 0 1 0 0 1 0 1 0]
after preprocess dimension:  (25000, 100)


In [31]:
from keras.models import Sequential

# Embedding 用來將文字 降維
from keras.layers import Dense, Embedding

# LSTM 是RNN的一種
from keras.layers import LSTM

model = Sequential()

# 將input的10000維變成128維
model.add(Embedding(10000,128))

model.add(LSTM(150))

model.add(Dense(1,activation='sigmoid'))

model.summary()

'''
LSTM Param的 167400  : 

step 1. 3(gates)*(128(上層)+150(鄰居)+1(bias))*150個cell = 125550
        gates : 控制資料流量通過多少 0~1
step 2. (128+150+1)*150 = 41850
step 3. 125550 + 41850 = 167400
'''

model.compile(loss='binary_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

model.fit(x_train,y_train,batch_size=32,epochs=15)

model.save('myRNN.h5')

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, None, 128)         1280000   
_________________________________________________________________
lstm_8 (LSTM)                (None, 150)               167400    
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 151       
Total params: 1,447,551
Trainable params: 1,447,551
Non-trainable params: 0
_________________________________________________________________
Epoch 1/15
25000/25000 [==============================] - 116s 5ms/step - loss: 0.4162 - accuracy: 0.8098
Epoch 2/15
25000/25000 [==============================] - 109s 4ms/step - loss: 0.2670 - accuracy: 0.8936
Epoch 3/15
25000/25000 [==============================] - 109s 4ms/step - loss: 0.1948 - accuracy: 0.9240
Epoch 4/15
25000/25000 [==============================] 

In [6]:
from keras.models import load_model
model = load_model('myRNN.h5')

In [7]:
score = model.evaluate(x_test,y_test)

25000/25000 [==============================] - 39s 2ms/step


In [9]:
print('loss',score[0])
print('accuracy',score[1])

loss 1.0401277062165737
accuracy 0.8274400234222412
